In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geocoder
import reverse_geocoder as rg
from math import sin, cos, sqrt, atan2, radians

In [4]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
train['isTrain'] = True
test['isTrain'] = False
data = train.append(test, sort=False)
data.head()

,id,atm_group,address,address_rus,lat,long,target,isTrain
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.011500,True
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.029710,True
2,8533.0,32.0,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",46.959413,142.741113,0.009540,True
3,8684.0,32.0,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",55.805827,37.515146,-0.094035,True
4,37.0,32.0,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",69.343541,88.211228,0.079277,True


In [5]:
def get_coords(x, y):
    if pd.isnull(x) | pd.isnull(y):
        return 'nan'
    else:
        coordinates = (x, y)
        results = rg.search(coordinates)
        return results

In [6]:
new_data = data.copy()

In [7]:
new_data['test'] = new_data.apply(lambda x: get_coords(x.lat, x.long), axis=1)
new_data.to_csv('new_data.csv')
new_data.head()

Loading formatted geocoded file...


,id,atm_group,address,address_rus,lat,long,target,isTrain,test
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.011500,True,"[{'lat': '46.95407', 'lon': '142.73603', 'name..."
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.029710,True,"[{'lat': '46.95407', 'lon': '142.73603', 'name..."
2,8533.0,32.0,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",46.959413,142.741113,0.009540,True,"[{'lat': '46.95407', 'lon': '142.73603', 'name..."
3,8684.0,32.0,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",55.805827,37.515146,-0.094035,True,"[{'lat': '55.8', 'lon': '37.51667', 'name': 'S..."
4,37.0,32.0,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",69.343541,88.211228,0.079277,True,"[{'lat': '69.3535', 'lon': '88.2027', 'name': ..."


In [8]:
new_df = pd.read_csv('new_data.csv')
df = new_df.copy()

In [96]:
df = df.reset_index().drop('index', 1)
df['city_lat'] = df[~df.address_rus.isnull()].test.apply(lambda x: x.split(',')[1])
df['city_long'] = df[~df.address_rus.isnull()].test.apply(lambda x: x.split(',')[3])
df['geo_city'] = df[~df.address_rus.isnull()].test.apply(lambda x: x.split(',')[5])
df['region'] = df[~df.address_rus.isnull()].test.apply(lambda x: x.split(',')[7])
df['city_lat'] = df['city_lat'].astype(str).apply(lambda x: x.replace("'", ''))
df['city_lat'] = df['city_lat'].astype(str).apply(lambda x: x.replace(")", ''))
df['city_long'] = df['city_long'].astype(str).apply(lambda x: x.replace("'", ''))
df['city_long'] = df['city_long'].astype(str).apply(lambda x: x.replace(")", ''))
df['geo_city'] = df['geo_city'].astype(str).apply(lambda x: x.replace("'", ''))
df['geo_city'] = df['geo_city'].astype(str).apply(lambda x: x.replace(")", ''))
df['region'] = df['region'].astype(str).apply(lambda x: x.replace("'", ''))
df['region'] = df['region'].astype(str).apply(lambda x: x.replace(")", ''))
df['city_lat'] = df['city_lat'].astype(float)
df['city_long'] = df['city_long'].astype(float)

In [10]:
R = 6373.0
def distance(x1,y1,x2,y2):    
    dlon = y2 - y1
    dlat = x2 - x1
    a = sin(dlat/2)**2 + cos(x1) * cos(x2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

In [16]:
df['diff_from_centre'] = df[~df.address_rus.isnull()].apply(lambda x: distance(x.lat, x.long, x.city_lat, x.city_long), axis=1)
df = df.drop('test', 1)
df.to_csv('data_with_geo.csv')